# Covid-19 현황 분석

In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

In [2]:
key_fd = open('gov_data_api_key.txt', mode='r')
govapi_key = key_fd.read(100)
key_fd.close()

In [3]:
start_date = '20200901'
end_date = '20201130'

In [4]:
base_url = 'http://openapi.data.go.kr/openapi/service/rest/Covid19/getCovid19SidoInfStateJson'
url = f'{base_url}?ServiceKey={govapi_key}&pageNo=1&numOfRows=10&startCreateDt={start_date}&endCreateDt={end_date}'

In [5]:
result = requests.get(url)
soup = BeautifulSoup(result.text, 'xml')
soup.find('resultCode').get_text()

'00'

In [6]:
soup.find('totalCount').string

'1883'

In [7]:
items = soup.find_all('item')
regions, qur_rates, std_days, death_cnts, inc_decs, isol_clr_cnts, def_cnts, local_occ_cnts = [], [], [], [], [], [], [], []
for item in items:
    death_cnt = item.find('deathCnt').string
    def_cnt = item.find('defCnt').string
    region = item.find('gubun').string
    inc_dec = item.find('incDec').string
    isol_clr_cnt = item.find('isolClearCnt').string
    local_occ_cnt = item.find('localOccCnt').string 
    qur_rate = item.find('qurRate').string
    std_day = item.find('stdDay').string
    regions.append(region); qur_rates.append(qur_rate); std_days.append(std_day); death_cnts.append(death_cnt) 
    inc_decs.append(inc_dec); isol_clr_cnts.append(isol_clr_cnt); def_cnts.append(def_cnt); local_occ_cnts.append(local_occ_cnt)

In [8]:
covid_status = pd.DataFrame({
    '지역': regions,
    '일시': std_days,
    '누적확진': def_cnts,
    '국내발생': local_occ_cnts,
    '전일대비 증감': inc_decs,
    '10만명당 발생률': qur_rates,
    '사망': death_cnts,
    '격리해제': isol_clr_cnts
})
covid_status.head(10)

,지역,일시,누적확진,국내발생,전일대비 증감,10만명당 발생률,사망,격리해제
0,검역,2020년 11월 30일 00시,21301,0,10,-,0,1618
1,검역,2020년 11월 30일 00시,2130,0,10,-,0,1618
2,제주,2020년 11월 30일 00시,80,0,1,11.93,0,60
3,경남,2020년 11월 30일 00시,625,19,20,18.59,1,388
4,경북,2020년 11월 30일 00시,1704,4,4,64.00,57,1560
5,전남,2020년 11월 30일 00시,424,3,4,22.74,2,214
6,전북,2020년 11월 30일 00시,334,16,16,18.38,2,173
7,충남,2020년 11월 30일 00시,899,4,5,42.36,9,667
8,충북,2020년 11월 30일 00시,327,22,22,20.45,3,207
9,강원,2020년 11월 30일 00시,652,8,8,42.32,6,400


In [9]:
# 검역 삭제
covid_status['누적확진'] = pd.to_numeric(covid_status['누적확진'])
covid_status['국내발생'] = pd.to_numeric(covid_status['국내발생'])
covid_status['전일대비 증감'] = pd.to_numeric(covid_status['전일대비 증감'])
covid_status['10만명당 발생률'] = pd.to_numeric(covid_status['10만명당 발생률'])
covid_status['사망'] = pd.to_numeric(covid_status['사망'])
covid_status['격리해제'] = pd.to_numeric(covid_status['격리해제'])

ValueError: Unable to parse string "-" at position 0

In [10]:
pd.pivot_table(covid_status, index=['일시', '지역'])

국내발생     누적확진  전일대비 증감
일시                지역                         
2020년 09월 01일 00시 강원    4.0    193.0      4.0
                  검역    0.0   1358.0      4.0
                  경기   60.0   3323.0     63.0
                  경남    3.0    239.0      3.0
                  경북    5.0   1464.0      9.0
...                     ...      ...      ...
2020년 11월 30일 00시 전북   16.0    334.0     16.0
                  제주    0.0     80.0      1.0
                  충남    4.0    899.0      5.0
                  충북   22.0    327.0     22.0
                  합계  414.0  43786.5    438.0

[1729 rows x 3 columns]